# Hyperparameter Optimization using H2O

This notebook compares the performance of H2O’s grid search, randomized grid search, and AutoML for hyperparameter optimization of the H2ORandomForestEstimator model.

## Overview
The key steps involve performing grid search, randomized grid search, and using H2O AutoML to find the best hyperparameters and model performance.

## Procedure
- **Grid Search**:
  - Perform grid search for H2ORandomForestEstimator with specified hyperparameters.
  - Display sorted grid results and evaluate the best model's performance.
- **Randomized Grid Search**:
  - Perform hyperparameter optimization using randomized grid search.
  - Display sorted results and evaluate the best model's performance.
- **H2O AutoML**:
  - Use H2O AutoML to find the best deep learning model for the classification task.
  - Display the leaderboard and evaluate the best model's performance.

References:
- [H2O Documentation](https://docs.h2o.ai)
- [allyears2k Headers Dataset](https://www.h2o.ai/resources/datasets/allyears2k_headers.zip)

In [5]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o import estimators
h2o.init()

# import the airlines dataset:
# This dataset is used to classify whether a flight will be delayed 'YES' or not "NO"
# original data can be found at http://www.transtats.bts.gov/
airlines= h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/airlines/allyears2k_headers.zip")

# convert columns to factors
airlines["Year"]= airlines["Year"].asfactor()
airlines["Month"]= airlines["Month"].asfactor()
airlines["DayOfWeek"] = airlines["DayOfWeek"].asfactor()
airlines["Cancelled"] = airlines["Cancelled"].asfactor()
airlines['FlightNum'] = airlines['FlightNum'].asfactor()

# set the predictor names and the response column name
predictors = ["Origin", "Dest", "Year", "UniqueCarrier",
              "DayOfWeek", "Month", "Distance", "FlightNum"]
response = "IsDepDelayed"

# split into train val and test
train, valid, test = airlines.split_frame(ratios=[.8, .1], seed=1234)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,8 mins 26 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_darien_b05g38
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.925 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


# 5.1.a

In [8]:
from h2o.grid.grid_search import H2OGridSearch

rf_estimator = estimators.H2ORandomForestEstimator()

ntrees = [10, 30, 50, 100]
max_depth = [1, 2, 4, 6]

hyper_parameters = {"ntrees": [10, 30, 50, 100], "max_depth": [1, 2, 4, 6]}

# Train the grid
rf_grid = H2OGridSearch(model=rf_estimator, hyper_params=hyper_parameters)
rf_grid.train(x=predictors, 
              y=response, 
              training_frame=train, 
              validation_frame=valid,
)


drf Grid Build progress: |

███████████████████████████████████████████████████████| (done) 100%


,max_depth,ntrees,model_ids,logloss
,6.0,30.0,Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_8,0.6110924
,6.0,100.0,Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_16,0.6114624
,6.0,50.0,Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_12,0.6117272
,6.0,10.0,Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_4,0.6155700
,4.0,100.0,Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_15,0.6264860
,4.0,50.0,Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_11,0.6270048
,4.0,30.0,Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_7,0.6274358
,4.0,10.0,Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_3,0.6320038
,2.0,10.0,Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_2,0.6505406
,2.0,50.0,Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_10,0.6506954


# 5.1.b

In [91]:
# Get the grid results
grid_results = rf_grid.get_grid(sort_by='accuracy', decreasing=True)
print(grid_results)


Hyper-Parameter Search Summary: ordered by decreasing accuracy
    max_depth    ntrees    model_ids                                                         accuracy
--  -----------  --------  ----------------------------------------------------------------  ----------
    6            50        Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_12  0.671596
    6            30        Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_8   0.668075
    6            100       Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_16  0.66784
    6            10        Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_4   0.66784
    4            50        Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_11  0.657277
    4            100       Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_15  0.656338
    4            30        Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_7   0.655634
    4            10

# 5.1.c

In [94]:
best_model = grid_results.models[0]

performance = best_model.model_performance(test)

print(f"Best Model ID: {best_model.model_id}")
print(f"Accuracy: {performance.accuracy()[0][1]:.4f}")
print(f"AUC score: {performance.auc():.4f}")


Best Model ID: Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_12
Accuracy: 0.6781
AUC score: 0.7380


```text
Best Model ID: Grid_DRF_py_40_sid_a544_model_python_1712505565922_3059_model_12
Accuracy: 0.6781
AUC score: 0.7380
```

The best model in the grid search is a decision tree model with an accuracy of 0.6781 and an AUC score of 0.7380.

## 5.2 Randomized grid search

# 5.2.a

In [21]:
# %% [markdown]
# # 5.2.a

# %%


rf_estimator = estimators.H2ORandomForestEstimator()

ntrees = [10, 30, 50, 100]
max_depth = [1, 2, 4, 6]

hyper_parameters = {"ntrees": [10, 30, 50, 100], "max_depth": [1, 2, 4, 6]}

search_criteria = {'strategy': 'RandomDiscrete', 'max_models': 10, 'seed': 9232}

# Train the grid
rf_grid2 = H2OGridSearch(model=rf_estimator, hyper_params=hyper_parameters, search_criteria=search_criteria)
rf_grid2.train(x=predictors, 
              y=response, 
              training_frame=train, 
              validation_frame=valid,
)



rf_grid2.train(
    x=predictors,
    y=response,
    training_frame=train,
    validation_frame=valid,
)



drf Grid Build progress: |

███████████████████████████████████████████████████████| (done) 100%
drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
Hyper-Parameter Search Summary: ordered by decreasing accuracy
    max_depth    ntrees    model_ids                                                         accuracy
--  -----------  --------  ----------------------------------------------------------------  ----------
    6            100       Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_1   0.670423
    6            30        Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_9   0.665962
    4            50        Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_10  0.659624
    4            10        Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_4   0.655164
    4            30        Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_5   0.655164
    4            100       Grid_DRF_py_40_sid_a544_model_


# 5.2.b


In [22]:

# %%
# Get the random grid results
random_grid2_results = rf_grid2.get_grid(sort_by='accuracy', decreasing=True)
print(random_grid2_results)

Hyper-Parameter Search Summary: ordered by decreasing accuracy
    max_depth    ntrees    model_ids                                                         accuracy
--  -----------  --------  ----------------------------------------------------------------  ----------
    6            100       Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_1   0.670423
    6            30        Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_9   0.665962
    4            50        Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_10  0.659624
    4            10        Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_4   0.655164
    4            30        Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_5   0.655164
    4            100       Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_7   0.654695
    2            30        Grid_DRF_py_40_sid_a544_model_python_1712505565922_6060_model_3   0.64108
    2            1

# 3. H2O AutoML


#  5.3.a


In [23]:

# %%
from h2o.automl import H2OAutoML

# Initialize the AutoML object
aml = H2OAutoML(max_models=20, seed=9232)

# Train the AutoML
aml.train(x=predictors, y=response, training_frame=train, validation_frame=valid)


AutoML progress: |
12:39:44.888: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
12:39:44.890: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████Job request failed Unexpected HTTP error: ('Connection aborted.', BadStatusLine('GET /3/Jobs/$03017f00000132d4ffffffff$_a460fe8ed666c9228a7e3806744456b HTTP/1.1\r\n')), will retry after 3s.
| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/20
# GBM base models (used / total),0/10
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),2/7
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [24]:
import pickle 
with open('aml.pkl', 'wb') as f:
    pickle.dump(aml, f)


# 5.3.b


In [60]:
# leaderboard
import pandas as pd 
pd.set_option('display.max_rows', None)
aml_lb = h2o.automl.get_leaderboard(aml)
aml_leaderboard = aml.leaderboard.sort(by='auc', ascending=False)
# display(aml_leaderboard.head(50))
print(aml_lb)

best_model = aml

model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse       mse
StackedEnsemble_AllModels_1_AutoML_1_20240407_123944     0.756087   0.585832  0.767857                0.348354  0.447694  0.20043
StackedEnsemble_BestOfFamily_1_AutoML_1_20240407_123944  0.755393   0.586386  0.767101                0.339776  0.447966  0.200674
GBM_1_AutoML_1_20240407_123944                           0.752938   0.58934   0.76335                 0.351937  0.449191  0.201773
GBM_3_AutoML_1_20240407_123944                           0.751078   0.590976  0.760174                0.347079  0.449897  0.202407
GBM_2_AutoML_1_20240407_123944                           0.749332   0.592415  0.759057                0.353812  0.450614  0.203053
GBM_4_AutoML_1_20240407_123944                           0.746035   0.596445  0.755082                0.357029  0.452407  0.204672
GBM_5_AutoML_1_20240407_123944                           0.745896   0.595113  0.7556

### Best AML Model and AUC

In [80]:
from pprint import pprint 
best_model = aml.leader

print(f"Best model: {best_model.model_id}")
print(f"AUC score: {best_model.auc():.4f}")



Best model: StackedEnsemble_AllModels_1_AutoML_1_20240407_123944
AUC score: 0.9109


### Full Parameters

In [84]:
display(best_model)
display(best_model.params)

key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/20
# GBM base models (used / total),0/10
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),2/7
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'StackedEnsemble_AllModels_1_AutoML_1_20240407_123944',
   'type': 'Key<Model>',
   'URL': '/3/Models/StackedEnsemble_AllModels_1_AutoML_1_20240407_123944'},
  'input': None},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'AutoML_1_20240407_123944_training_py_40_sid_a544',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/AutoML_1_20240407_123944_training_py_40_sid_a544'},
  'input': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'AutoML_1_20240407_123944_training_py_40_sid_a544',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/AutoML_1_20240407_123944_training_py_40_sid_a544'}},
 'response_column': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    


# 5.3.c


In [85]:
# * (c) Display the AUC score of the best model for the test set. (2)

performance = best_model.model_performance(test)
print(f"AUC score (test set): {performance.auc():.4f}")


AUC score (test set): 0.7452


# 5.3.d

No extreme gradient boosting models were trained by the AutoML model. Therefore, I have adjusted part d to identify the best gradient boosting model.

In [90]:
# * (d) Identify the best XGBoost model among all the models tested using log loss as the criteria. (2)

aml_lb = h2o.automl.get_leaderboard(aml, extra_columns = 'ALL').sort(by='logloss', ascending=True)
display(aml_lb.head(49))

# xgb = aml.get_best_model(algorithm="xgboost", criterion="logloss")
gbm = aml.get_best_model(algorithm="gbm", criterion="logloss")

print(f"Best gbm model: {gbm.model_id}")
print(f"Log loss: {gbm.logloss():.4f}")

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_1_AutoML_1_20240407_123944,0.756087,0.585832,0.767857,0.348354,0.447694,0.20043,10583,0.119953,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_1_20240407_123944,0.755393,0.586386,0.767101,0.339776,0.447966,0.200674,6984,0.110318,StackedEnsemble
GBM_1_AutoML_1_20240407_123944,0.752938,0.58934,0.76335,0.351937,0.449191,0.201773,876,0.011968,GBM
GBM_3_AutoML_1_20240407_123944,0.751078,0.590976,0.760174,0.347079,0.449897,0.202407,527,0.010315,GBM
GBM_2_AutoML_1_20240407_123944,0.749332,0.592415,0.759057,0.353812,0.450614,0.203053,490,0.01101,GBM
GBM_5_AutoML_1_20240407_123944,0.745896,0.595113,0.755672,0.355188,0.451947,0.204256,434,0.010734,GBM
GBM_4_AutoML_1_20240407_123944,0.746035,0.596445,0.755082,0.357029,0.452407,0.204672,857,0.010363,GBM
GBM_grid_1_AutoML_1_20240407_123944_model_2,0.743288,0.596842,0.754765,0.362023,0.45291,0.205128,385,0.011329,GBM
GBM_grid_1_AutoML_1_20240407_123944_model_1,0.741982,0.598302,0.75391,0.359084,0.453623,0.205774,535,0.014209,GBM
XRT_1_AutoML_1_20240407_123944,0.743518,0.602369,0.750886,0.35747,0.455421,0.207408,3657,0.020476,DRF


Best gbm model: GBM_1_AutoML_1_20240407_123944
Log loss: 0.5104


As mentioned above, no extreme gradient boosting models were trained by the AutoML model, so gradient boosting models are considered in-place. The best gradient boosting model with respect to log loss is: `GBM_1_AutoML_1_20240407_123944`
```text
Best gbm model: GBM_1_AutoML_1_20240407_123944
Log loss: 0.5104
```
